#### torch-jixie

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch.utils.data
import math
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim     # for constructing optimizer
from typing import Optional
import time
from sklearn.preprocessing import StandardScaler
from kernels.flash_atten_full_int8 import attention  # 导入Triton的attention函数

def quant_pertoken(X):
    X_max, _ = torch.abs(X).max(dim=-1)
    X_scale = X_max / 127
    ret = torch.round(X / X_scale[:, :, :, None]).to(torch.int8)
    return ret, X_scale

def quant_pertensor(X):
    X_max, _ = torch.abs(X).max(dim=-1)
    X_max, _ = torch.max(X_max, dim=-1)
    X_scale = X_max / 127
    ret = torch.round(X / X_scale[:, :, None, None]).to(torch.int8)
    return ret, X_scale

class EnsureContiguousGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        return x
    
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.contiguous()

class JXdataset(Dataset):
    def __init__(self, path, scaler=None, is_training=True):
        JX = pd.read_csv(path)
        JX_data = JX.values
        
        features = JX_data[:, :-1]
        targets = JX_data[:, -1]
        
        # NOTE: must normalize features or the model output will be nan
        if is_training:
            self.scaler = StandardScaler()
            self.features = self.scaler.fit_transform(features)
        else:
            self.features = scaler.transform(features)
        
        self.features = torch.tensor(self.features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
    
    def __getitem__(self, index):
        return self.features[index], self.targets[index]

    def __len__(self):
        return len(self.features)

# MultiheadAttention
class MHA(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality
        qkv = self.c_attn(x)
        # 使用chunk更清晰地将张量分成三等份
        q, k, v = qkv.chunk(3, dim=2)
        
        # 重塑为多头格式 [B, nh, T, hs]
        head_size = self.n_embd // self.n_head
        k = k.view(B, T, self.n_head, head_size).transpose(1, 2)
        q = q.view(B, T, self.n_head, head_size).transpose(1, 2)
        v = v.view(B, T, self.n_head, head_size).transpose(1, 2)

        # 计算注意力
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(head_size))
        att = F.softmax(att, dim=-1)
        y = att @ v
        
        # 重塑回原始形状
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # 输出投影
        y = self.c_proj(y)
        return y

class Net(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=256, n_head=8, seq_len=256):
        super().__init__()
        assert hidden_dim % n_head == 0
        self.config = type('Config', (), {
            'n_embd': hidden_dim,
            'n_head': n_head
        })
        
        self.input_embedding = nn.Linear(input_dim, hidden_dim)
        self.mha = MHA(self.config)
        self.output_layer = nn.Linear(hidden_dim, 1)
        self.seq_len = seq_len

    def forward(self, x):
        x = x.unsqueeze(1).expand(-1, self.seq_len, -1)  # [B, seq_len, input_dim]
        x = self.input_embedding(x)  # [B, seq_len, hidden_dim]
        x = self.mha(x)              # [B, seq_len, hidden_dim]
        x = self.output_layer(x)     # [B, seq_len, 1]
        x = x[:, 0, :] # 取第一个时间步（等价于 cls token）
        return x.squeeze(-1).squeeze(-1)

class TritonMHA(nn.Module):
    def __init__(self, hidden_dim, n_head, seq_len=256):
        super().__init__()
        assert hidden_dim % n_head == 0
        head_dim = hidden_dim // n_head
        assert head_dim % 16 == 0 and head_dim >= 32

        self.hidden_dim = hidden_dim
        self.n_head = n_head
        self.seq_len = seq_len
        self.head_dim = head_dim

        # 添加 qkv projection 和 output projection
        self.c_attn = nn.Linear(hidden_dim, 3 * hidden_dim)
        self.c_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        batch_size = x.size(0)

        # 1. QKV projection
        qkv = self.c_attn(x)  # [B, T, 3 * hidden_dim]
        q, k, v = qkv.chunk(3, dim=2)  # 分成三份

        # 2. reshape: [B, n_head, T, head_dim]
        q = q.view(batch_size, self.seq_len, self.n_head, self.head_dim).transpose(1, 2).contiguous()
        k = k.view(batch_size, self.seq_len, self.n_head, self.head_dim).transpose(1, 2).contiguous()
        v = v.view(batch_size, self.seq_len, self.n_head, self.head_dim).transpose(1, 2).contiguous()

        # 3. Attention
        sm_scale = 1.0 / math.sqrt(self.head_dim)
        # 首先需要对q, k, v进行量化
        q_int8, q_scale = quant_pertoken(q)  # 按token量化q
        k_int8, k_scale = quant_pertoken(k)  # 按token量化k
        v_int8, v_scale = quant_pertensor(v) # 按tensor量化v

        # 然后调用attention_full_int8，传入所有所需参数
        x = attention(q_int8, k_int8, v_int8, q_scale, k_scale, v_scale, True, sm_scale)
        x = EnsureContiguousGrad.apply(x)
        x = x.transpose(1, 2).contiguous().view(batch_size, self.seq_len, self.hidden_dim)

        # 4. Output projection
        x = self.c_proj(x)
        return x

class NetWithTriton(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=256, n_head=8, seq_len=256):
        super().__init__()
        assert hidden_dim % n_head == 0

        self.seq_len = seq_len

        self.input_embedding = nn.Linear(input_dim, hidden_dim)
        self.mha = TritonMHA(hidden_dim=hidden_dim, n_head=n_head, seq_len=seq_len)
        self.output_layer = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = x.unsqueeze(1).expand(-1, self.seq_len, -1)  # [B, seq_len, input_dim]
        x = self.input_embedding(x)                      # [B, seq_len, hidden_dim]
        x = self.mha(x)                                  # [B, seq_len, hidden_dim]
        x = self.output_layer(x)                         # [B, seq_len, 1]
        x = x[:, 0, :]                                   # 取第一个时间步
        return x.squeeze(-1)

def convert_pytorch_to_triton_model(pytorch_model):
    """
    Converts PyTorch MHA-based model to Triton version, including qkv and proj weights
    """
    triton_model = NetWithTriton(
        input_dim=pytorch_model.input_embedding.in_features,
        hidden_dim=pytorch_model.input_embedding.out_features,
        n_head=pytorch_model.mha.n_head,
        seq_len=256  # 确保与 Triton 要求一致
    )

    # 1. 拷贝输入嵌入层
    triton_model.input_embedding.load_state_dict(pytorch_model.input_embedding.state_dict())

    # 2. 拷贝 MHA 的 qkv 和 proj 层
    triton_model.mha.c_attn.load_state_dict(pytorch_model.mha.c_attn.state_dict())
    triton_model.mha.c_proj.load_state_dict(pytorch_model.mha.c_proj.state_dict())

    # 3. 拷贝输出层
    triton_model.output_layer.load_state_dict(pytorch_model.output_layer.state_dict())

    return triton_model
     

ModuleNotFoundError: No module named 'sklearn'

#### sas-jixie

In [1]:
import swat
import getpass

sas_server_name = 'vfl-022.engage.sas.com'
port_number = '443'

sas_server_auth = '/SASLogon/oauth/authorize?client_id=SWAT&response_type=code'
print('Please navigate to the link below to retreive your authentication code:\n'+ '\x1b[0;1;30;43m' + 'https://'  + sas_server_name + sas_server_auth + '\x1b[0m')

Please navigate to the link below to retreive your authentication code:
https://vfl-022.engage.sas.com/SASLogon/oauth/authorize?client_id=SWAT&response_type=code


In [2]:
authcode = getpass.getpass('Authorization Code: ')

In [3]:
conn_string = 'https://' + sas_server_name + ':' + port_number + '/cas-shared-default-http'
conn = swat.CAS(conn_string, authcode=authcode)
print(conn)

CAS('vfl-022.engage.sas.com', 443, protocol='https', name='py-session-1', session='5bd38292-ede8-6540-9cc0-6c20677549ed')


In [ ]:
# !pip install git+https://github.com/sassoftware/python-dlpy.git

In [4]:
import swat
import os
import numpy as np
import pandas as pd
import sys
import dlpy
from dlpy import Sequential
from dlpy import *
from dlpy.model import TextParms
from dlpy.blocks import Bidirectional
from dlpy.applications import TextClassification
from dlpy.network import *
from dlpy.utils import *
from dlpy.applications import *
from dlpy.model import *
from dlpy.images import *
from dlpy.layers import *


conn.loadTable(path='jixie_train_data.csv', casout={'name': 'jixie_train_data', 'caslib': 'casuser'}, importOptions={'fileType': 'csv'})
tb = conn.CASTable('jixie_train_data', caslib='casuser')
tb.shape

NOTE: Cloud Analytic Services made the file jixie_train_data.csv available as table JIXIE_TRAIN_DATA in caslib CASUSER(22321320@zju.edu.cn).


(426, 5)

In [5]:
df = tb.head()
df = (tb.f3.value_counts())
display(type(df), df)
# df.plot(kind='bar', figsize=(8, 6))

pandas.core.series.Series

5400.0    86
3600.0    85
4050.0    85
4500.0    85
4950.0    85
dtype: int64

In [6]:
# 查看数据集结构
print("训练数据的列名:", tb.columns.tolist())

# 定义输入和目标变量
input_vars = tb.columns[:-1].tolist()
target_var = tb.columns[-1]
print("输入变量:", input_vars)
print("目标变量:", target_var)


model = Sequential(conn, model_table='MHA_Regression_Simple')

model.add(InputLayer(n_channels=4, name='input_layer'))
model.add(Dense(n=32, act='relu', name='dense2'))
model.add(MultiHeadAttention(n=32, n_attn_heads=2, name='mha_layer'))
model.add(Dense(n=16, act='relu', name='pre_output'))
model.add(OutputLayer(n=1, act='IDENTITY', name='output_layer'))

# 编译模型
model.compile()

# 训练模型
# model.fit(
#     data=tb,
#     inputs=input_vars,
#     target=target_var,
#     mini_batch_size=16,
#     max_epochs=30,
#     log_level=2,
#     lr=0.05
# )

训练数据的列名: ['f1', 'f2', 'f3', 'f4', 'tg']
输入变量: ['f1', 'f2', 'f3', 'f4']
目标变量: tg
NOTE: Input layer added.
NOTE: Fully-connected layer added.
NOTE: Multi-head attention layer added.
NOTE: Fully-connected layer added.
NOTE: Output layer added.
NOTE: Model compiled successfully.
NOTE: Model compiled successfully.


In [7]:
# 查看模型的所有层
print(model.layers[0])
print(model.layers[1])
print(model.layers[2])
print(model.layers[3])
print(model.layers[4])

input_layer
dense2
mha_layer
pre_output
output_layer


In [8]:
import functools
import time
import types
import pprint
from tqdm import tqdm
import swat
import dlpy
from dlpy import Sequential
from dlpy.model import *
from dlpy.layers import *
from dlpy.utils import *

# 创建一个字典来存储调用次数
call_count = {}

def count_calls(func, module_name=None):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        full_name = module_name + '.' + func.__name__
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        
        elapsed_time = end_time - start_time
        
        if full_name not in call_count:
            call_count[full_name] = {"count": 0, "total_time": 0.0}
        
        call_count[full_name]["count"] += 1
        call_count[full_name]["total_time"] += elapsed_time
        
        return result
    wrapper_count_calls._is_decorated = True
    return wrapper_count_calls

def set_new_attr(module, attr_name, attr):
    if not hasattr(attr, "_is_decorated"):
        decorated_attr = count_calls(attr, module.__name__)
        decorated_attr._is_decorated = True
        setattr(module, attr_name, decorated_attr)

# 递归封装所有的包
def auto_decorate_module(module, visited=None):
    if visited is None:
        visited = set()
    
    module_name = module.__name__
    if module_name in visited:
        return
    visited.add(module_name)
    for attr_name in dir(module):
        try:
            attr = getattr(module, attr_name)
            if isinstance(attr, types.FunctionType):
                set_new_attr(module, attr_name, attr)
            elif isinstance(attr, types.ModuleType) and attr.__name__.startswith('dlpy'):
                auto_decorate_module(attr, visited)
            elif isinstance(attr, type):
                auto_decorate_class(attr)
            elif callable(attr):
                set_new_attr(module, attr_name, attr)
        except AttributeError:
            continue

def auto_decorate_class(cls):
    for attr_name in dir(cls):
        try:
            attr = getattr(cls, attr_name)
            if isinstance(attr, types.FunctionType):
                set_new_attr(cls, attr_name, attr)
            elif attr_name in ['__add__', '__mul__', '__sub__', '__truediv__', '__matmul__', '__pow__', '__mod__']:
                set_new_attr(cls, attr_name, attr)
        except (AttributeError, TypeError):
            continue

# 自动装饰 dlpy 模块及其子模块
auto_decorate_module(dlpy)
# 装饰 Layer 类的 __call__ 方法
Layer.__call__ = count_calls(Layer.__call__, 'Layer')

# 装饰 MultiHeadAttention.__call__
from dlpy.layers import MultiHeadAttention
# 同样先清理标记
if hasattr(MultiHeadAttention.__call__, "_is_decorated"):
    del MultiHeadAttention.__call__._is_decorated
MultiHeadAttention.__call__ = count_calls(MultiHeadAttention.__call__,
                                          'MultiHeadAttention')

In [9]:
batch_size = 128
seq_len = 256
for index, row in tb.head(batch_size).iterrows():
    # 创建输入张量
    input_data = row.values[:4]
    input_seq = np.tile(input_data, (seq_len, 1))
    input_tensor = Tensor(InputLayer(256, 4))
    input_tensor.shape = (256, 4)
    input_tensor._value = input_data

    # 前向计算并记录时间
    output1 = model.layers[0](input_tensor)
    dense_output = model.layers[1](output1)
    mha_output = model.layers[2](dense_output)
    pre_output = model.layers[3](mha_output)
    output = model.layers[4](pre_output)

pprint.pprint(call_count)

{'CASTable.__deepcopy__': {'count': 2, 'total_time': 0.0005397796630859375},
 'CASTable.__getattr__': {'count': 1, 'total_time': 0.006033182144165039},
 'CASTable.__init__': {'count': 2, 'total_time': 0.00045299530029296875},
 'CASTable.__setattr__': {'count': 32, 'total_time': 7.987022399902344e-05},
 'CASTable._fetch': {'count': 1, 'total_time': 0.7064030170440674},
 'CASTable._retrieve': {'count': 1, 'total_time': 0.7049293518066406},
 'CASTable._sample': {'count': 1, 'total_time': 1.6689300537109375e-06},
 'CASTable._use_casout_for_stat': {'count': 2,
                                   'total_time': 7.05718994140625e-05},
 'CASTable.copy': {'count': 1, 'total_time': 0.0003101825714111328},
 'CASTable.get_action_params': {'count': 1,
                                'total_time': 5.0067901611328125e-06},
 'CASTable.get_connection': {'count': 3, 'total_time': 7.867813110351562e-06},
 'CASTable.get_fetch_params': {'count': 2,
                               'total_time': 3.0994415283203

#### torch-guangdian

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch.utils.data
import math
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim     # for constructing optimizer
from typing import Optional
import time
from kernels.quantize_gemm_int8 import *
from kernels.conv2d import conv2d_triton

class GDdataset(Dataset):
    def __init__(self, path):
        self.data = pd.read_csv(path)
        SFR = torch.tensor(self.data.iloc[:, 2:46].values)
        blocks = [SFR[:, i:i+4].reshape(-1, 2, 2) for i in range(0, 45, 5)] # 将36列中的每4列合并成一个2*2矩阵，得到9个块
        
        self.value = torch.cat([torch.cat(blocks[i:i+3], dim=2) for i in range(0, 9, 3)], dim=1) # 将9个块按3*3的方式拼成一个大矩阵
        self.value = self.value.unsqueeze(1).to(torch.float32)
        
        self.target = torch.tensor([1.0 if x == 'OK' else 0.0 for x in self.data.iloc[:, 48].values])
        self.target = self.target.unsqueeze(1)
        
    def __getitem__(self, index):
        return self.value[index], self.target[index]
        
        
    def __len__(self):
        return len(self.data)
    
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         # 卷积层
#         # input:(batch_size, 1, 6, 6), output:(batch_size, num_kernels, 3, 3)
#         self.conv = nn.Sequential(
#             nn.Conv2d(in_channels=1, out_channels=4096, kernel_size=2, stride=2),
#             nn.BatchNorm2d(num_features=4096),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=1),
#         )
        
#         # 全连接层
#         self.fc = nn.Sequential(
#             nn.Linear(in_features=4096*3*3, out_features=8192),
#             nn.ReLU(),
#             nn.Linear(in_features=8192, out_features=4096),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             nn.Linear(in_features=4096, out_features=2048),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             nn.Linear(in_features=2048, out_features=1024),
#             nn.ReLU(),
#             nn.Linear(in_features=1024, out_features=256),
#             nn.ReLU(),
#             nn.Linear(in_features=256, out_features=1),
#         )
        
#         self.sigmoid = nn.Sigmoid()
        
        
#     def forward(self, x):
#         x = self.conv(x)
#         # x = x.view(x.shape[0], -1)  # (batch_size, 4096*3*3)
#         B = x.shape[0]
#         Cin = x.shape[1] * x.shape[2] * x.shape[3]  # 计算出 Cin = 4096 * 3 * 3
#         T = 1  # 设置 T = 1，因为不涉及序列长度，这里作为占位符
#         x = x.view(B, T, Cin)
#         x = self.fc(x)
#         x = x.view(x.shape[0], -1)    # (batch_size, 1)
#         x = self.sigmoid(x)     
#         return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层
        # input:(batch_size, 1, 6, 6), output:(batch_size, num_kernels, 3, 3)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1),
        )
        
        # 全连接层
        self.fc = nn.Sequential(
            nn.Linear(in_features=16*3*3, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            # nn.Dropout(p=0.5),
            nn.Linear(in_features=32, out_features=1),
        )
        
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        x = self.conv(x)
        # x = x.view(x.shape[0], -1)  # (batch_size, 4096*3*3)
        B = x.shape[0]
        Cin = x.shape[1] * x.shape[2] * x.shape[3]  # 计算出 Cin = 4096 * 3 * 3
        T = 1  # 设置 T = 1，因为不涉及序列长度，这里作为占位符
        x = x.view(B, T, Cin)
        x = self.fc(x)
        x = x.view(x.shape[0], -1)    # (batch_size, 1)
        x = self.sigmoid(x)     
        return x

class TritonLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
        super().__init__()
        factory_kwargs = {'device': device, 'dtype': dtype}
        self.in_features = in_features
        self.out_features = out_features
        
        # 初始化权重 (out_features, in_features)
        self.weight = nn.Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        
        if bias:
            self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        
        # 预计算量化权重
        self.register_buffer('quantized_weight', None)
        self.register_buffer('weight_scale', None)
        
        self.reset_parameters()
        self.quantize_weight()  # 初始化时预量化权重
    
    def reset_parameters(self):
        # 标准的 PyTorch 初始化方法
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            nn.init.uniform_(self.bias, -bound, bound)
    
    def quantize_weight(self):
        # 转置权重以匹配 matmul 需求：从 (out_features, in_features) 到 (in_features, out_features)
        weight_t = self.weight.t().contiguous()
        # 按列量化（相当于原始权重的行）
        quantized_weight, weight_scale = quantize_int8(weight_t, axis=0)
        self.quantized_weight = quantized_weight
        self.weight_scale = weight_scale
    
    def forward(self, input):
        # 如果处于训练模式或权重尚未量化，则量化权重
        if self.training or self.quantized_weight is None:
            self.quantize_weight()
        
        # 处理输入
        original_shape = input.shape
        if input.dim() > 2:
            # 将高维输入重塑为 2D
            input = input.reshape(-1, input.size(-1))
        
        # 量化输入
        quantized_input, input_scale = quantize_int8_perrow(input)
        
        # 执行量化矩阵乘法
        output = matmul_int8(quantized_input, input_scale, 
                           self.quantized_weight, self.weight_scale)
        
        # 添加偏置（如果存在）
        if self.bias is not None:
            output = output + self.bias
        
        # 恢复原始维度（如果需要）
        if len(original_shape) > 2:
            output_shape = original_shape[:-1] + (self.out_features,)
            output = output.reshape(output_shape)
        
        return output

class Conv2dTriton(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True, device=None, dtype=None):
        super(Conv2dTriton, self).__init__()
        
        # Handle kernel_size as int or tuple
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)
        
        # Currently, the Triton implementation only supports stride = kernel_size
        if isinstance(stride, int):
            stride = (stride, stride)
        
        assert stride == kernel_size, "The Triton implementation only supports stride == kernel_size"
        assert padding == 0, "The Triton implementation doesn't support padding"
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        
        # Initialize weights and bias
        self.weight = nn.Parameter(torch.empty((out_channels, in_channels, kernel_size[0], kernel_size[1])))
        if bias:
            self.bias = nn.Parameter(torch.empty(out_channels))
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters()
        
    def reset_parameters(self):
        # Standard initialization as in PyTorch
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            nn.init.uniform_(self.bias, -bound, bound)
    
    def forward(self, x):
        # Ensure input dimensions are divisible by kernel dimensions
        batch_size, channels, height, width = x.shape
        assert height % self.kernel_size[0] == 0 and width % self.kernel_size[1] == 0, \
            f"Input height ({height}) and width ({width}) should be divisible by kernel dimensions ({self.kernel_size})"
        
        # Use the Triton implementation
        bias_tensor = self.bias if self.bias is not None else torch.zeros(self.out_channels, device=x.device, dtype=x.dtype)
        output = conv2d_triton(x, self.weight, bias_tensor)
        
        # Ensure output has the same dtype as input
        return output.to(x.dtype)


# class NetWithTriton(nn.Module):
#     def __init__(self):
#         super(NetWithTriton, self).__init__()
#         # Replace nn.Conv2d with Conv2dTriton
#         self.conv = nn.Sequential(
#             Conv2dTriton(in_channels=1, out_channels=4096, kernel_size=2, stride=2),
#             nn.BatchNorm2d(num_features=4096),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=1),
#         )
        
#         # Use TritonLinear for fully connected layers
#         self.fc = nn.Sequential(
#             TritonLinear(in_features=4096*3*3, out_features=8192),
#             nn.ReLU(),
#             TritonLinear(in_features=8192, out_features=4096),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             TritonLinear(in_features=4096, out_features=2048),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             TritonLinear(in_features=2048, out_features=1024),
#             nn.ReLU(),
#             TritonLinear(in_features=1024, out_features=256),
#             nn.ReLU(),
#             TritonLinear(in_features=256, out_features=1),
#         )
        
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x):
#         x = self.conv(x)
#         B = x.shape[0]
#         Cin = x.shape[1] * x.shape[2] * x.shape[3]  # Calculate Cin = 4096 * 3 * 3
#         T = 1  # Set T = 1 as placeholder
#         x = x.view(B, T, Cin)
#         x = self.fc(x)
#         x = x.view(x.shape[0], -1)  
#         x = self.sigmoid(x)     
#         return x

class NetWithTriton(nn.Module):
    def __init__(self):
        super(NetWithTriton, self).__init__()
        # Replace nn.Conv2d with Conv2dTriton
        self.conv = nn.Sequential(
            Conv2dTriton(in_channels=1, out_channels=16, kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1),
        )
        
        # Use TritonLinear for fully connected layers
        self.fc = nn.Sequential(
            TritonLinear(in_features=16*3*3, out_features=256),
            nn.ReLU(),
            TritonLinear(in_features=256, out_features=64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            TritonLinear(in_features=64, out_features=32),
            nn.ReLU(),
            # nn.Dropout(p=0.5),
            TritonLinear(in_features=32, out_features=1),
        )
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.conv(x)
        B = x.shape[0]
        Cin = x.shape[1] * x.shape[2] * x.shape[3]  # Calculate Cin = 4096 * 3 * 3
        T = 1  # Set T = 1 as placeholder
        x = x.view(B, T, Cin)
        x = self.fc(x)
        x = x.view(x.shape[0], -1)  
        x = self.sigmoid(x)     
        return x

def convert_pytorch_to_triton_model(pytorch_model):
    """
    Convert standard PyTorch model to a model using TritonLinear and Conv2dTriton
    """
    triton_model = NetWithTriton()
    
    # Copy Conv2d weights to Conv2dTriton
    pytorch_conv = pytorch_model.conv[0]  # Get the Conv2d layer
    triton_conv = triton_model.conv[0]    # Get the Conv2dTriton layer
    
    # Copy weights and bias
    print(f"Copying Conv weights: PyTorch {pytorch_conv.weight.shape} -> Triton {triton_conv.weight.shape}")
    triton_conv.weight.data.copy_(pytorch_conv.weight.data)
    if hasattr(pytorch_conv, 'bias') and pytorch_conv.bias is not None:
        triton_conv.bias.data.copy_(pytorch_conv.bias.data)
    
    # Copy other layers in the conv sequential (BatchNorm, etc.)
    for i in range(1, len(pytorch_model.conv)):
        print(f"Copying layer: {type(pytorch_model.conv[i]).__name__}")
        triton_model.conv[i].load_state_dict(pytorch_model.conv[i].state_dict())
    
    # Copy linear layer weights to TritonLinear
    pytorch_linears = [m for m in pytorch_model.fc if isinstance(m, nn.Linear)]
    triton_linears = [m for m in triton_model.fc if isinstance(m, TritonLinear)]
    
    assert len(pytorch_linears) == len(triton_linears), \
        f"Number of Linear layers doesn't match: PyTorch {len(pytorch_linears)} vs Triton {len(triton_linears)}"
    
    for i, (pytorch_linear, triton_linear) in enumerate(zip(pytorch_linears, triton_linears)):
        print(f"Copying Linear {i+1} weights: PyTorch {pytorch_linear.weight.shape} -> Triton {triton_linear.weight.shape}")
        triton_linear.weight.data.copy_(pytorch_linear.weight.data)
        if pytorch_linear.bias is not None:
            triton_linear.bias.data.copy_(pytorch_linear.bias.data)
        
        # Update quantized weights
        triton_linear.quantize_weight()
    
    return triton_model
    

In [2]:
from call_count import call_count, monitor, reset_counters, print_statistics
import time
import types

def run_inference(batch_sizes=[64, 128, 256, 512], num_batches=5):
    """
    Run inference on Triton models and measure operator-level performance
    """
    print("Starting inference test with real dataset...")
    
    # Define device
    device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
    torch.cuda.set_device(device)  # Set default device
    torch.cuda.empty_cache()  # Clear unused GPU memory
    
    # Try to load pre-trained model
    try:
        model_state_dict = torch.load("model.pt")
        print("Successfully loaded pre-trained model weights")
        pytorch_model = Net()
        pytorch_model.load_state_dict(model_state_dict)
    except:
        print("Could not load pre-trained model, using default weights")
        pytorch_model = Net()
    # pytorch_model = Net()
    
    # Convert model to use Triton implementations
    pytorch_model = pytorch_model.to(device).half()
    triton_model = convert_pytorch_to_triton_model(pytorch_model)
    triton_model = triton_model.to(device).half()
    
    # Set to evaluation mode
    triton_model.eval()
    
    # Add profiling for custom operators
    def profile_method(instance, method_name, display_name):
        """Profile a method call and record time in call_count."""
        original_method = getattr(instance, method_name)
        
        def profiled_method(*args, **kwargs):
            start_time = time.time()
            result = original_method(*args, **kwargs)
            torch.cuda.synchronize()
            end_time = time.time()
            
            elapsed_time = end_time - start_time
            
            if display_name not in call_count:
                call_count[display_name] = {"count": 0, "total_time": 0.0}
            
            call_count[display_name]["count"] += 1
            call_count[display_name]["total_time"] += elapsed_time
            
            return result
        
        setattr(instance, method_name, profiled_method)
        return original_method
    
    # Store original methods to restore later
    originals = []
    
    # Profile the conv and fc sequential containers
    originals.append((triton_model.conv, "forward", 
                    profile_method(triton_model.conv, "forward", "triton_model.conv")))
    originals.append((triton_model.fc, "forward", 
                    profile_method(triton_model.fc, "forward", "triton_model.fc")))
    
    # Profile specific Conv2dTriton layers
    for i, layer in enumerate(triton_model.conv):
        if isinstance(layer, Conv2dTriton):
            originals.append((layer, "forward", 
                            profile_method(layer, "forward", f"Conv2dTriton[{i}]")))
    
    # Profile specific TritonLinear layers
    for i, layer in enumerate(triton_model.fc):
        if isinstance(layer, TritonLinear):
            originals.append((layer, "forward", 
                            profile_method(layer, "forward", f"TritonLinear[{i}]")))
    
    # Create the dataset once (we'll reuse it for different batch sizes)
    try:
        train_dataset = GDdataset("./train_data.csv")
        print(f"Successfully loaded dataset with {len(train_dataset)} samples")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return
    
    # Test for different batch sizes
    for batch_size in batch_sizes:
        print(f"\n==== Batch Size: {batch_size} ====")
        
        # Store batches for testing - manually shuffle the dataset indices
        indices = torch.randperm(len(train_dataset), device='cpu').tolist()
        test_batches = []
        batch_count = 0
        
        for i in range(0, min(len(indices), batch_size * num_batches), batch_size):
            if batch_count >= num_batches:
                break
                
            # Get a batch of indices
            batch_indices = indices[i:i+batch_size]
            if len(batch_indices) < batch_size:
                # Skip incomplete batches
                continue
                
            # Collect inputs and targets for this batch
            batch_inputs = []
            batch_targets = []
            for idx in batch_indices:
                inp, tgt = train_dataset[idx]
                batch_inputs.append(inp)
                batch_targets.append(tgt)
            
            # Stack the tensors into a batch
            inputs = torch.stack(batch_inputs).to(device).half()
            targets = torch.stack(batch_targets).to(device).half()
            
            test_batches.append((inputs, targets))
            batch_count += 1
        
        # Ensure we have enough batches
        if len(test_batches) < num_batches:
            print(f"Warning: Could only extract {len(test_batches)} batches instead of {num_batches}")
            num_actual_batches = len(test_batches)
        else:
            num_actual_batches = num_batches
        
        # Verify data shape
        print(f"Input shape: {test_batches[0][0].shape}")
        
        # Warmup
        for i in range(min(10, len(test_batches))):
            inputs, _ = test_batches[i % len(test_batches)]
            with torch.no_grad():
                _ = triton_model(inputs)
        torch.cuda.synchronize()
        
        # Reset counters before measurement
        reset_counters()
        
        # Triton model inference with operator-level profiling
        with monitor():
            for i in range(num_actual_batches):
                inputs, targets = test_batches[i]
                with torch.no_grad():
                    triton_output = triton_model(inputs)
                torch.cuda.synchronize()
        
        # Print operator-level statistics
        print("\nOperator-level performance statistics")
        print_statistics()
        
        # Print sample outputs for the last batch
        batch_to_print = min(batch_size, triton_output.size(0))
        print(f"\nSample output (first {min(5, batch_to_print)} from last batch):")
        for i in range(min(5, batch_to_print)):
            print(f"  Sample {i}: Output={triton_output[i].item():.4f}")
    
    # Restore original methods
    for instance, method_name, original in originals:
        setattr(instance, method_name, original)

In [3]:
run_inference(batch_sizes=[128], num_batches=5)

Starting inference test with real dataset...


/tmp/ipykernel_2519564/2227982509.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load("model.pt")


Could not load pre-trained model, using default weights
Copying Conv weights: PyTorch torch.Size([16, 1, 2, 2]) -> Triton torch.Size([16, 1, 2, 2])
Copying layer: BatchNorm2d
Copying layer: ReLU
Copying layer: MaxPool2d
Copying Linear 1 weights: PyTorch torch.Size([256, 144]) -> Triton torch.Size([256, 144])
Copying Linear 2 weights: PyTorch torch.Size([64, 256]) -> Triton torch.Size([64, 256])
Copying Linear 3 weights: PyTorch torch.Size([32, 64]) -> Triton torch.Size([32, 64])
Copying Linear 4 weights: PyTorch torch.Size([1, 32]) -> Triton torch.Size([1, 32])
Successfully loaded dataset with 8773 samples

==== Batch Size: 128 ====
Input shape: torch.Size([128, 1, 6, 6])


/home/gaozhe/workspace/suanzi/triton/call_count.py:157: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  attr = getattr(cls, attr_name)



Operator-level performance statistics
Function Name                                      Count      Total Time (s)  Avg Time (s)   
------------------------------------------------------------------------------------------
OpOverloadPacket.__str__                           1928       0.001986        0.000001       
torch._C._get_operation_overload                   964        0.005771        0.000006       
OpOverloadPacket.__getattr__                       964        0.000962        0.000001       
Module.__getattr__                                 130        0.000096        0.000001       
Mapping.get                                        130        0.000253        0.000002       
torch.cuda._cuda_isInBadFork                       125        0.000031        0.000000       
torch.cuda.is_initialized                          125        0.000167        0.000001       
torch.cuda._lazy_init                              125        0.000279        0.000002       
torch._C._cuda_getDevice

#### sas-guangdian

In [1]:
import swat
import getpass

sas_server_name = 'vfl-022.engage.sas.com'
port_number = '443'

sas_server_auth = '/SASLogon/oauth/authorize?client_id=SWAT&response_type=code'
print('Please navigate to the link below to retreive your authentication code:\n'+ '\x1b[0;1;30;43m' + 'https://'  + sas_server_name + sas_server_auth + '\x1b[0m')

Please navigate to the link below to retreive your authentication code:
https://vfl-022.engage.sas.com/SASLogon/oauth/authorize?client_id=SWAT&response_type=code


In [2]:
authcode = getpass.getpass('Authorization Code: ')

In [3]:
conn_string = 'https://' + sas_server_name + ':' + port_number + '/cas-shared-default-http'
conn = swat.CAS(conn_string, authcode=authcode)
print(conn)

CAS('vfl-022.engage.sas.com', 443, protocol='https', name='py-session-1', session='2f846508-19f3-824d-9a7a-d1010aae8170')


In [4]:
import swat
import os
import numpy as np
import pandas as pd
import sys
import dlpy
import functools
import time
import types
import pprint
from dlpy import Sequential
from dlpy import *
from dlpy.model import TextParms
from dlpy.blocks import Bidirectional
from dlpy.applications import TextClassification
from dlpy.network import *
from dlpy.utils import *
from dlpy.applications import *
from dlpy.model import *
from dlpy.images import *
from dlpy.layers import *


conn.loadTable(path='guangdian_train_data.csv', casout={'name': 'guangdian_train_data', 'caslib': 'casuser'}, importOptions={'fileType': 'csv'})
tb = conn.CASTable('guangdian_train_data', caslib='casuser')
tb.shape

NOTE: Cloud Analytic Services made the file guangdian_train_data.csv available as table GUANGDIAN_TRAIN_DATA in caslib CASUSER(22321320@zju.edu.cn).


(8773, 52)

In [5]:
SFR = tb.iloc[:, 2:46].values
blocks = [SFR[:, i:i+4].reshape(-1, 2, 2) for i in range(0, 45, 5)] # 将36列中的每4列合并成一个2*2矩阵，得到9个块
x_train = np.concatenate([np.concatenate(blocks[i:i+3], axis=2) for i in range(0, 9, 3)], axis=1) # 将9个块按3*3的方式拼成一个大矩阵
# 修改train_data
# 将 numpy 数组转换为 pandas DataFrame
X_train_flat = x_train.reshape(8773, -1)  # 展平图像数据
df_train = pd.DataFrame(X_train_flat)
df_train['label'] = tb.SFR_Result
df_train['label'] = df_train['label'].map({'OK': 1, 'NG': 0})
train_data = conn.upload_frame(df_train, casout={'name':'train_data', 'replace':True})

NOTE: Cloud Analytic Services made the uploaded file available as table TRAIN_DATA in caslib CASUSER(22321320@zju.edu.cn).
NOTE: The table TRAIN_DATA has been created in caslib CASUSER(22321320@zju.edu.cn) from binary data uploaded to Cloud Analytic Services.


In [7]:
# 模型定义和训练
model = Sequential(conn, model_table='Simple_CNN')
model.add(InputLayer(1, 6, 6))
model.add(Conv2d(n_filters=16, width=2, height=2, stride=2, act='relu'))
model.add(BatchNormalization())
model.add(Pooling(1))
model.add(Dense(16*3*3, act='relu'))
model.add(Dense(256, act='relu'))
model.add(Dense(64, act='relu'))
model.add(Dense(32, act='relu'))
model.add(Dense(1))
model.add(OutputLayer(act='sigmoid', n=1))

# model = Sequential(conn, model_table='Simple_CNN')
# model.add(InputLayer(1, 6, 6))
# model.add(Conv2d(n_filters=4096, width=2, height=2, stride=2, act='relu'))
# model.add(BatchNormalization())
# model.add(Pooling(1))
# model.add(Dense(4096*3*3, act='relu'))
# model.add(Dense(8192, act='relu'))
# model.add(Dense(4096, act='relu'))
# model.add(Dense(2048, act='relu'))
# model.add(Dense(1024, act='relu'))
# model.add(Dense(256, act='relu'))
# model.add(Dense(1))
# model.add(OutputLayer(act='sigmoid', n=1))

input_vars = train_data.columns[:-1].tolist()  # 除去 'label' 列的所有列名
target_var = 'label'
model.fit(
    data=train_data,
    inputs=input_vars,
    target=target_var,
    mini_batch_size=128,
    max_epochs=100,
    lr = 0.1,
    n_threads=1,
    log_level=2
)


<html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>Microsoft-Azure-Application-Gateway/v2</center>
</body>
</html>

/home/gaozhe/anaconda3/envs/triton/lib/python3.10/site-packages/dlpy/network.py:1948: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if layer.type is not 'input':
/home/gaozhe/anaconda3/envs/triton/lib/python3.10/site-packages/dlpy/network.py:1948: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if layer.type is not 'input':
/home/gaozhe/anaconda3/envs/triton/lib/python3.10/site-packages/dlpy/network.py:1948: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if layer.type is not 'input':
/home/gaozhe/anaconda3/envs/triton/lib/python3.10/site-packages/dlpy/network.py:1948: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if layer.type is not 'input':


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

##### benchmark

In [24]:
# 创建一个字典来存储调用次数
call_count = {}

def count_calls(func, module_name=None):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        full_name = module_name + '.' + func.__name__
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        
        elapsed_time = end_time - start_time
        
        if full_name not in call_count:
            call_count[full_name] = {"count": 0, "total_time": 0.0}
        
        call_count[full_name]["count"] += 1
        call_count[full_name]["total_time"] += elapsed_time
        
        return result
    wrapper_count_calls._is_decorated = True
    return wrapper_count_calls

def set_new_attr(module, attr_name, attr):
    if not hasattr(attr, "_is_decorated"):
        decorated_attr = count_calls(attr, module.__name__)
        decorated_attr._is_decorated = True
        setattr(module, attr_name, decorated_attr)

# 递归封装所有的包
def auto_decorate_module(module, visited=None):
    if visited is None:
        visited = set()
    
    module_name = module.__name__
    if module_name in visited:
        return
    visited.add(module_name)
    for attr_name in dir(module):
        try:
            attr = getattr(module, attr_name)
            if isinstance(attr, types.FunctionType):
                set_new_attr(module, attr_name, attr)
            elif isinstance(attr, types.ModuleType) and attr.__name__.startswith('dlpy'):
                auto_decorate_module(attr, visited)
            elif isinstance(attr, type):
                auto_decorate_class(attr)
            elif callable(attr):
                set_new_attr(module, attr_name, attr)
        except AttributeError:
            continue

def auto_decorate_class(cls):
    for attr_name in dir(cls):
        try:
            attr = getattr(cls, attr_name)
            if isinstance(attr, types.FunctionType):
                set_new_attr(cls, attr_name, attr)
            elif attr_name in ['__add__', '__mul__', '__sub__', '__truediv__', '__matmul__', '__pow__', '__mod__']:
                set_new_attr(cls, attr_name, attr)
        except (AttributeError, TypeError):
            continue

# 自动装饰 dlpy 模块及其子模块
auto_decorate_module(dlpy)
# 装饰 Layer 类的 __call__ 方法
Layer.__call__ = count_calls(Layer.__call__, 'Layer')

In [25]:
batch_size=128
# 遍历 train_data 中的batch行进行前向计算
for index, row in df_train.head(batch_size).iterrows():
    # 从 test_data 中获取当前行的前 36 个元素，并将其重塑为 6x6 矩阵
    input_data = row.values[:36].reshape((1, 1, 6, 6))
    
    # 创建输入张量
    input_tensor = Tensor(InputLayer(1, 6, 6))
    input_tensor.shape = (1, 1, 6, 6)
    input_tensor._value = input_data

    # 前向计算并记录时间
    conv_output = model.layers[1](input_tensor)
    batch_norm_output = model.layers[2](conv_output)
    pooling_output = model.layers[3](batch_norm_output)
    dense_output1 = model.layers[4](pooling_output)
    dense_output2 = model.layers[5](dense_output1)
    dense_output3 = model.layers[6](dense_output2)
    dense_output4 = model.layers[7](dense_output3)
    dense_output5 = model.layers[8](dense_output4)
    output = model.layers[9](dense_output5)

pprint.pprint(call_count)

{'BN.__call__': {'count': 128, 'total_time': 0.0017046928405761719},
 'BN._assert_inputs': {'count': 128, 'total_time': 7.414817810058594e-05},
 'Conv2d.__call__': {'count': 128, 'total_time': 0.002118825912475586},
 'Conv2d._assert_inputs': {'count': 128, 'total_time': 0.0001163482666015625},
 'Dense.__call__': {'count': 640, 'total_time': 0.0071752071380615234},
 'Dense._assert_inputs': {'count': 640, 'total_time': 0.00035572052001953125},
 'InputLayer.__init__': {'count': 128, 'total_time': 0.0036852359771728516},
 'Layer.__call__': {'count': 1536, 'total_time': 0.026607990264892578},
 'Layer.__init__': {'count': 128, 'total_time': 0.0003139972686767578},
 'Layer._assert_inputs': {'count': 256, 'total_time': 0.00020241737365722656},
 'Node.__init__': {'count': 1152, 'total_time': 0.0007941722869873047},
 'Tensor.__init__': {'count': 1408, 'total_time': 0.0009775161743164062},
 'dlpy.layers._unpack_config': {'count': 128,
                                'total_time': 0.00108742713928